In [ ]:
# # Building Name & State Dimensions & Control Dimensions & Num. of Constraints & Main Objective (Minimize) \\
# from energym.wrappers.rl_wrapper import StableBaselinesRLWrapper
# from buildings_factory import *
# objectives = ["average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "power demand",
#               "power demand",
#               "CO2 emission",
#               "CO2 emission",
#               "power demand",
#               "power demand",
#               "power demand",
#               "power demand"]

# model_info = ""
# for building_name in buildings_list:
#     # min_kpis, max_kpis = collect_baseline_kpi(building_name)
#     building_idx = buildings_list.index(building_name)
#     env = get_env(building_name)
#     inputs = get_inputs(building_name, env)
#     default_control = default_controls[building_idx]
#     env_rl = StableBaselinesRLWrapper(building_name, None, None, reward_func)
#     input_dim = env_rl.observation_space.shape[0]
#     control_dim = env_rl.action_space.shape[0]
#     constraint_num = 0
#     for key, val in env.kpi_options.items():
#         if val["type"] == 'avg_dev': constraint_num += 1 
#     main_objective = objectives[building_idx]
#     model_info += f"{building_name} & {input_dim} & {control_dim} & {constraint_num} & {main_objective} \\ "
# print(model_info)

In [7]:
import pandas as pd
# buildings_list = ["ApartmentsThermal-v0", "ApartmentsGrid-v0", "Apartments2Thermal-v0",
#                   "Apartments2Grid-v0", "OfficesThermostat-v0", "MixedUseFanFCU-v0",
#                   "SeminarcenterThermostat-v0", "SeminarcenterFull-v0", "SimpleHouseRad-v0",
#                   "SimpleHouseRSla-v0", "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"] 

plot_building_names = ["SimpleHouseRad-v0"]
seeds = [7]
# "MOPO_ori_simulator"
model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "MOPO_ori_simulator", "SAC_bs_simulator", "SAC_dnn_predictor"]
legend_names = ["TD3PlusBC", "CQL", "MOPO", "Best", "Ours"]
res_cols = ["manual_baseline_eval_episode_reward", "manual_eval_episode_reward"]
            
colors = ['r', 'g', 'r--', 'g--']

res_data = []
for seed in seeds:
    for model_name, legend_name in zip(model_names, legend_names):
        for building in plot_building_names:
            res_loc = f"data/models/{building}/{model_name}_seed{seed}/logs/best/rewards.csv"
            res_row = pd.read_csv(res_loc).iloc[-1, :]
            res_data.append([seed, building, model_name, legend_name,
                             res_row["eval_episode_reward"], res_row["manual_eval_episode_reward"],
                             res_row["baseline_eval_episode_reward"], res_row["manual_baseline_eval_episode_reward"]])
res_df = pd.DataFrame(res_data, columns=["seed", "building", "model_name", 
                                         "legend", "learnt_reward", "manual_reward",
                                         "learnt_baseline_reward", "manual_baseline_reward"])
            

res_df.head()

,seed,building,model_name,legend,learnt_reward,manual_reward,learnt_baseline_reward,manual_baseline_reward
0,7,SimpleHouseRad-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-177483.466627,-177483.466627,-17615.840744,-17615.840744
1,7,SimpleHouseRad-v0,CQL_ori_simulator,CQL,-112654.314497,-112654.314497,-17614.399906,-17614.399906
2,7,SimpleHouseRad-v0,MOPO_ori_simulator,MOPO,-23254.272436,-23254.272436,-17615.535772,-17615.535772
3,7,SimpleHouseRad-v0,SAC_bs_simulator,Best,28873.410687,2868.498587,23906.326374,-11436.503063
4,7,SimpleHouseRad-v0,SAC_dnn_predictor,Ours,99050.380078,1592.266724,3997.866507,-11436.655500


In [9]:
import numpy as np
cols = res_df.columns.tolist()
# "RL_reward", "manual_RL_reward", "baseline_reward", "manual_baseline_reward"
cols.remove("seed")
stat_df = res_df[cols].groupby(["building", "model_name", "legend"]).agg(learnt_reward_mean=("learnt_reward", np.mean),
                                                           learnt_reward_std=("learnt_reward", np.std),
                                                           learnt_baseline_reward_mean=("learnt_baseline_reward", np.mean),
                                                           learnt_baseline_reward_std=("learnt_baseline_reward", np.std),
                                                           manual_reward_mean=("manual_reward", np.mean),
                                                           manual_reward_std=("manual_reward", np.std),
                                                           manual_baseline_reward_mean=("manual_baseline_reward", np.mean),
                                                           manual_baseline_reward_std=("manual_baseline_reward", np.std)).reset_index()

stat_df.fillna(0.0, inplace=True)
stat_df

,building,model_name,legend,learnt_reward_mean,learnt_reward_std,learnt_baseline_reward_mean,learnt_baseline_reward_std,manual_reward_mean,manual_reward_std,manual_baseline_reward_mean,manual_baseline_reward_std
0,SimpleHouseRad-v0,CQL_ori_simulator,CQL,-112654.314497,0.0,-17614.399906,0.0,-112654.314497,0.0,-17614.399906,0.0
1,SimpleHouseRad-v0,MOPO_ori_simulator,MOPO,-23254.272436,0.0,-17615.535772,0.0,-23254.272436,0.0,-17615.535772,0.0
2,SimpleHouseRad-v0,SAC_bs_simulator,Best,28873.410687,0.0,23906.326374,0.0,2868.498587,0.0,-11436.503063,0.0
3,SimpleHouseRad-v0,SAC_dnn_predictor,Ours,99050.380078,0.0,3997.866507,0.0,1592.266724,0.0,-11436.655500,0.0
4,SimpleHouseRad-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-177483.466627,0.0,-17615.840744,0.0,-177483.466627,0.0,-17615.840744,0.0


In [10]:
bs_stat_df = stat_df[['building', "manual_baseline_reward_mean"]].groupby("building").mean().reset_index()
bs_stat_df.columns = ['building', 'manual_reward_mean']
bs_stat_df['legend'] = 'baseline'
bs_stat_df['manual_reward_std'] = 0.0

In [15]:
cols = ["building", "legend", "manual_reward_mean", "manual_reward_std"]
stat_df_all = pd.concat([stat_df[cols], bs_stat_df[cols]])
stat_df_all['reward'] = stat_df_all.apply(lambda x: f"{x['manual_reward_mean']:.2f}\xb1{x['manual_reward_std']}", axis=1)
df = stat_df_all.pivot(index='building', columns='legend', values='reward').reset_index()
df

legend,building,Best,CQL,MOPO,Ours,TD3PlusBC,baseline
0,SimpleHouseRad-v0,2868.50±0.0,-112654.31±0.0,-23254.27±0.0,1592.27±0.0,-177483.47±0.0,-15143.79±0.0


In [17]:
print_cols = ["building", "baseline", "CQL", "TD3PlusBC", "MOPO", "Ours"]
print(" & ".join(print_cols))
for _, row in df.iterrows():
    print(" & ".join([str(row[col]) for col in print_cols]))

building & baseline & CQL & TD3PlusBC & MOPO & Ours
SimpleHouseRad-v0 & -15143.79±0.0 & -112654.31±0.0 & -177483.47±0.0 & -23254.27±0.0 & 1592.27±0.0
